In [1]:
from wiktionary_de_parser.dump_processor import WiktionaryDump


dump = WiktionaryDump(dump_dir_path="tmp")
dump.download_dump()

In [2]:
from wiktionary_de_parser import WiktionaryParser

parser = WiktionaryParser()
pages = []

for page in dump.pages():
    if page.redirect_to:
        continue

    # if page.name != "ordo":
    #     continue

    pages.append(page)

In [7]:
from tqdm.notebook import tqdm
from wiktionary_de_parser import WiktionaryParser
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

from wiktionary_de_parser.models import MeaningDict


def chunks(lst, n):
    """Split list into n chunks"""
    chunk_size = len(lst) // n + (1 if len(lst) % n else 0)
    for i in range(0, len(lst), chunk_size):
        yield lst[i : i + chunk_size]


def process_chunk(pages_chunk):
    local_dict = {}
    parser = WiktionaryParser()

    for page in pages_chunk:
        for entry in parser.entries_from_page(page):
            entry_parsed = parser.parse_entry(entry, include_meanings=True)

            if entry_parsed.meanings is None:
                continue

            local_dict[page.name] = entry_parsed.meanings

    return local_dict


# Get number of CPU cores (leave one free for system)
n_cores = max(1, mp.cpu_count() - 1)

# Split pages into chunks
page_chunks = list(chunks(pages, n_cores))

all_lists: dict[str, list[MeaningDict]] = {}
with ProcessPoolExecutor(max_workers=n_cores) as executor:
    # Process chunks in parallel with progress bar
    futures = list(
        tqdm(
            executor.map(process_chunk, page_chunks),
            total=len(page_chunks),
            desc="Processing chunks",
        )
    )

    # Merge results
    for result_dict in futures:
        all_lists.update(result_dict)


Processing chunks:   0%|          | 0/27 [00:00<?, ?it/s]

In [10]:
from wiktionary_de_parser.parser.parse_meanings import (
    format_meanings,
)


print(len(all_lists))

print(format_meanings(all_lists["ordo"]))

265321
• gegenständlich: Reihe, Sitzreihe (Reihe der Sitze oder Bänke im Theater), Lage, Schicht
• militärisch:
  • (abstrakt: Militär-Gruppierung, Legion) Glied, Zenturie, Abteilung, Abteilungsglied, Kompanie
  • (personifiziert) der Hauptmann selbst
• politisch-gesellschaftlich: Stand, Klasse, Rang, Stellung
• abstrakt: Ordnung, gehörige Reihenfolge, Anordnung, Regel
  • übertragen: Verfassung, Zustand
• kirchlich (mittellateinisch):
  • Ordensgemeinschaft von Mönchen
  • kirchlicher Stand: Weihestufe
  • die (göttliche) Weltordnung
• Taxonomie: Biologische Systematik (neulateinisch): Ordnung oder fachwissenschaftlich Ordo (Zusammenfassung mehrerer eng verwandter Familien, Teil eine Klasse)


In [11]:
# recursively calculate the depth of the wiki_list.sublist
from wiktionary_de_parser.models import MeaningDict
from wiktionary_de_parser.parser.parse_meanings import format_meaning_dict


def calculate_depth(list_item: MeaningDict):
    if not list_item.get("sublist"):
        return 0
    return 1 + max(
        [calculate_depth(subitem) for subitem in list_item.get("sublist", [])]
    )


# Calculate the number of maximum list depth in all lists in all_lists
max_depth = 0
for word, lists in all_lists.items():
    for wiki_list_item in lists:
        depth = calculate_depth(wiki_list_item)
        if depth > max_depth:
            max_depth = depth

        if depth == 3:
            print(f"word: {word}")
            print(format_meaning_dict(wiki_list_item))
            print("\n")

print(max_depth)

word: condicional
• Der Kondicional der katalanischen Sprache drückt die Möglichkeit von Tatsachen oder Aktionen aus, die implizit oder explizit von Umständen in der Vergangenheit abhängig sind. Er wird in folgenden Situationen verwendet:
  • Zum Ausdruck der Wahrscheinlichkeit des Eintretens eines Ereignisses in der Zukunft:
    • Si tinguessim més temps, acabaríem el projecte.
      • Mit mehr Zeit würden wir das Projekt erfolgreich beenden.
  • Als Ausdruck einer höflichen Bitte (condicional de cortesia):
    • Tindries la bondat d’ / Podries ajudar-me en aquestes negociacions ?
      • Wärest du so freundlich, mich bei diesen Verhandlungen zu unterstützen?
  • Bei einer kaschierten oder vorsichtig formulierten Aufforderung:
    • Podríeu treballar més acuradament!
      • Ihr könntet sorgfältiger arbeiten!
  • Bei einem höflich, zurückhaltend formulierten Wunsch:
    • M’agradaria fer un creuer.
      • Ich möchte gern einmal eine Kreuzfahrt machen.
  • In höflich, zurückhaltend fo